Paso 1: Abre el archivo de datos y estudia la información general
===============

In [ ]:
import pandas as pd

# Cargar el archivo CSV
file_path = "/datasets/games.csv"
games_data = pd.read_csv(file_path)

# Mostrar información general del dataset en una celda separada
games_data.info()
print(games_data.head())


Paso 2: Prepara los datos
===============
Reemplaza los nombres de las columnas (ponlos en minúsculas).

In [ ]:
# Normalizar los nombres de las columnas
games_data.columns = games_data.columns.str.lower()

- Convierte los datos en los tipos necesarios.

In [ ]:
# Manejo de tipos de datos
games_data['year_of_release'] = pd.to_numeric(games_data['year_of_release'], errors='coerce')
games_data['user_score'] = pd.to_numeric(games_data['user_score'], errors='coerce')

- Describe las columnas en las que los tipos de datos han sido cambiados y explica por qué.

year_of_release se ha convertido a numérico para facilitar el análisis temporal.

user_score se ha convertido a numérico para manejar valores como "TBD" y realizar cálculos.

- Si es necesario, elige la manera de tratar los valores ausentes.

In [ ]:
# Identificar valores ausentes
missing_data_summary = games_data.isnull().sum()

print(missing_data_summary)

In [ ]:
# Decisiones:
# 1. 'Name': Solo 2 valores nulos, eliminados ya que no afectan significativamente el análisis general.
games_data = games_data.dropna(subset=['name'])

# 2. 'Year_of_Release': Se rellena con el valor más frecuente (moda) para conservar la integridad temporal.
most_frequent_year = games_data['year_of_release'].mode()[0]
games_data['year_of_release'] = games_data['year_of_release'].fillna(most_frequent_year).astype(int)

# 3. 'Genre': Solo 2 valores nulos, se eliminan y se cambia a columna catgorica.
games_data['genre'] = games_data['genre'].astype('category')
games_data = games_data.dropna(subset=['genre'])

# 4. 'Critic_Score' y 'User_Score': Se rellenan con la mediana para evitar sesgos.
games_data['critic_score'] = games_data['critic_score'].fillna(games_data['critic_score'].median())
games_data['user_score'] = games_data['user_score'].fillna(games_data['user_score'].median())

# 5. 'Rating': Los valores nulos se reemplazan por "Unknown" para clasificar juegos sin información de rating.
games_data['rating'] = games_data['rating'].fillna('Unknown')

# Ver resumen actualizado de datos ausentes

print(games_data.isnull().sum())

In [ ]:
# Muestreo de datos limpios

games_data.info()
print(games_data.sample(5))

- Explica por qué rellenaste los valores ausentes como lo hiciste o por qué decidiste dejarlos en blanco.

Los valores "TBD" en user_score se manejaron como valores nulos (NaN) porque estos representan datos pendientes de determinar ("To Be Determined"). Dado que no ofrecen una información válida para análisis, se transformaron en valores ausentes para evitar interpretaciones erróneas y mantener la integridad de los datos.

- ¿Por qué crees que los valores están ausentes? Brinda explicaciones posibles.

Los valores ausentes pueden deberse a varias razones:

Limitaciones en la recopilación de datos:

Algunas columnas, como Critic_Score y User_Score, dependen de evaluaciones externas. Si un juego no fue ampliamente revisado, las puntuaciones pueden estar ausentes.
Los datos de Rating podrían faltar si el juego no recibió una clasificación ESRB formal.

Errores o inconsistencias en el registro:

Juegos más antiguos o menos populares podrían tener datos incompletos debido a prácticas de registro menos estrictas en su momento.
Falta de relevancia percibida:

Algunos juegos pueden haber sido lanzados en regiones específicas, haciendo que ciertos datos (como clasificaciones o puntuaciones) no fueran recolectados.

- Presta atención a la abreviatura TBD: significa "to be determined" (a determinar). Especifica cómo piensas manejar estos casos.


Los valores marcados como "TBD" en la columna user_score fueron tratados como valores ausentes (NaN) por las siguientes razones:

Naturaleza de TBD: La abreviatura "TBD" (to be determined) indica que estos datos no estaban disponibles al momento de la recopilación. Tratarlos como ausentes permite manejar su falta de información de forma adecuada en análisis futuros.

Evitar sesgos en imputaciones: Reemplazar "TBD" con una media, mediana o algún otro valor predeterminado podría introducir sesgos en los resultados, especialmente si los juegos con "TBD" tienen características distintas.

- Calcula las ventas totales (la suma de las ventas en todas las regiones) para cada juego y coloca estos valores en una columna separada.

In [ ]:
# Crear una columna de ventas totales
games_data['total_sales'] = games_data[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)

# Confirmar insercion de columna
print(games_data.sample(5))

Paso 3: Analiza los datos
===============
- Mira cuántos juegos fueron lanzados en diferentes años. ¿Son significativos los datos de cada período?

In [ ]:
import matplotlib.pyplot as plt

# Juegos lanzados por año
games_per_year = games_data['year_of_release'].value_counts().sort_index()

# Verificar la proporción de juegos por año
total_games = games_per_year.sum()
proportion_per_year = (games_per_year / total_games) * 100

# Gráfico de barras: juegos lanzados por año
plt.figure(figsize=(14, 7))
plt.bar(games_per_year.index, games_per_year.values, color='skyblue', alpha=0.85)
plt.title("Juegos Lanzados por Año", fontsize=16)
plt.xlabel("Año de Lanzamiento", fontsize=12)
plt.ylabel("Número de Juegos", fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Distribución por año:

El número de juegos lanzados anualmente creció desde la década de 1980, alcanzando un pico en 2008 con 1,696 juegos.
Posteriormente, el número de lanzamientos disminuyó gradualmente hasta 502 juegos en 2016.
Proporción por período:

La mayor actividad en lanzamientos ocurrió entre 2005 y 2011, representando el período de mayor contribución a los datos totales.
Los años anteriores a 2000 y posteriores a 2013 tienen una contribución significativamente menor.
Estos datos son significativos para análisis recientes, especialmente centrados en el período de auge entre 2005 y 2011

- Observa cómo varían las ventas de una plataforma a otra. Elige las plataformas con las mayores ventas totales y construye una distribución basada en los datos de cada año. Busca las plataformas que solían ser populares pero que ahora no tienen ventas. ¿Cuánto tardan generalmente las nuevas plataformas en aparecer y las antiguas en desaparecer?

In [ ]:
# Ventas totales por plataforma
platform_sales = games_data.groupby('platform')['total_sales'].sum().sort_values(ascending=False)

# Seleccionar las plataformas principales (mayores ventas totales)
top_platforms = platform_sales.head(5).index

# Ventas por año para plataformas principales
platform_trends = games_data[games_data['platform'].isin(top_platforms)].groupby(['year_of_release', 'platform'])['total_sales'].sum().unstack()

# Gráfico de tendencias de ventas por plataforma
plt.figure(figsize=(14, 8))
platform_trends.plot(kind='line', marker='o', figsize=(14, 8))
plt.title("Tendencia de Ventas por Plataforma (Top 5)", fontsize=16)
plt.xlabel("Año de Lanzamiento", fontsize=12)
plt.ylabel("Ventas Totales (millones de dólares)", fontsize=12)
plt.legend(title="Plataforma", fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Buscar plataformas que ya no tienen ventas
platform_last_year = games_data.groupby('platform')['year_of_release'].max()
inactive_platforms = platform_last_year[platform_last_year < games_data['year_of_release'].max()]



Observaciones sobre las ventas por plataforma

1. Plataformas con mayores ventas totales:
PS2: Lidera con 1,255.77 millones de dólares.
X360, PS3, Wii, DS: Completan el Top 5, todas con ventas superiores a 800 millones.

2. Distribución de ventas por año:
PS2: Tuvo su pico entre 2001 y 2005, desapareciendo hacia 2011.
X360 y PS3: Mostraron su auge entre 2008 y 2012.
Wii: Aumentó rápidamente desde su lanzamiento en 2006, pero decayó después de 2010.

3. Plataformas que ya no tienen ventas:
Ejemplos: NES, N64, SNES, PS2, PSP.
La mayoría desaparecieron antes de 2015, coincidiendo con la llegada de nuevas generaciones.

4. Ciclo de vida de plataformas:
En promedio, las plataformas tardan entre 5 y 10 años en alcanzar su auge y perder relevancia.
Nuevas plataformas suelen reemplazar a las antiguas rápidamente, como se ve con la transición de PS2 a PS3/PS4.

- Determina para qué período debes tomar datos. Para hacerlo mira tus respuestas a las preguntas anteriores. Los datos deberían permitirte construir un modelo para 2017.

Basándome en las observaciones previas:

Crecimiento y declive de plataformas:

El auge de las plataformas comenzó alrededor del año 2000 con la llegada de la generación de PS2, X360 y Wii.
A partir de 2015, la relevancia de plataformas más recientes como PS4 y Xbox One es notable.
Relevancia de lanzamientos:

Los lanzamientos alcanzaron su pico en 2008, y aunque han disminuido, los datos de 2012 en adelante reflejan transiciones importantes hacia plataformas actuales.
Ventas y comportamiento de mercado:

Las tendencias más relevantes para ventas en 2017 están representadas por datos de plataformas recientes (PS4, XOne) y géneros populares durante el período 2010-2016.
Período sugerido:
Datos desde 2010 hasta 2016: Este intervalo captura transiciones clave y proporciona una base sólida para predecir tendencias en 2017.

- Trabaja solo con los datos que consideras relevantes. Ignora los datos de años anteriores.

In [ ]:
# Filtrar datos relevantes desde 2010 hasta 2016
relevant_data = games_data[games_data['year_of_release'] >= 2010]

# Confirmar el rango de años en los datos filtrados
relevant_data['year_of_release'].value_counts().sort_index()

- ¿Qué plataformas son líderes en ventas? ¿Cuáles crecen y cuáles se reducen? Elige varias plataformas potencialmente rentables.

In [ ]:
# Ventas totales por plataforma en el período relevante (2010-2016)
platform_sales_relevant = relevant_data.groupby('platform')['total_sales'].sum().sort_values(ascending=False)

# Identificar las plataformas principales por ventas totales
top_platforms_relevant = platform_sales_relevant.head(5)

# Tendencias de crecimiento/reducción de las plataformas principales
platform_trends_relevant = relevant_data.groupby(['year_of_release', 'platform'])['total_sales'].sum().unstack()

# Calcular la diferencia en ventas año a año para identificar tendencias
platform_trends_growth = platform_trends_relevant.diff()

# Gráfico de líneas: Tendencia de ventas de las principales plataformas
top_platforms_names = top_platforms_relevant.index
platform_trends_top = platform_trends_relevant[top_platforms_names]

plt.figure(figsize=(14, 8))
for platform in top_platforms_names:
    plt.plot(platform_trends_top.index, platform_trends_top[platform], marker='o', label=platform)

plt.title("Tendencia de Ventas de las Principales Plataformas (2010-2016)", fontsize=16)
plt.xlabel("Año de Lanzamiento", fontsize=12)
plt.ylabel("Ventas Totales (millones de dólares)", fontsize=12)
plt.legend(title="Plataformas", fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

PS4: Muestra un crecimiento constante hasta 2015, destacándose como la plataforma con mayor potencial futuro.
PS3 y X360: Ambas plataformas experimentan un declive continuo en este período, indicando una transición hacia nuevas generaciones.
Wii: Las ventas caen drásticamente después de 2012.
3DS: Muestra un declive gradual, aunque mantiene cierta relevancia.

- Crea un diagrama de caja para las ventas globales de todos los juegos, desglosados por plataforma. ¿Son significativas las diferencias en las ventas? ¿Qué sucede con las ventas promedio en varias plataformas? Describe tus hallazgos.


In [ ]:
# Crear un diagrama de caja para las ventas globales por plataforma
plt.figure(figsize=(26, 5))
relevant_data.boxplot(column='total_sales', by='platform', grid=False, showfliers=False)
plt.title("Distribución de Ventas Globales por Plataforma (2010-2016)", fontsize=16)
plt.suptitle("")  # Eliminar el título automático
plt.xlabel("Plataforma", fontsize=12)
plt.ylabel("Ventas Totales (millones de dólares)", fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Calcular estadísticas descriptivas para las plataformas
sales_stats = relevant_data.groupby('platform')['total_sales'].describe()
sales_stats.sort_values(by='mean', ascending=False)

Plataformas con mayores ventas promedio:

X360 y PS4: Presentan medianas más altas y mayores valores máximos en comparación con otras plataformas.
PS3: También muestra ventas consistentes con un rango intercuartil relativamente amplio.
Variabilidad en ventas:

Las plataformas como X360, PS4, y PS3 tienen una alta dispersión, indicando una variabilidad significativa en el éxito de los juegos.
Plataformas como PSP y PSV tienen distribuciones más concentradas y valores mucho más bajos.
Outliers significativos:

Varias plataformas muestran outliers en ventas, lo que sugiere juegos extremadamente exitosos que impulsan los promedios hacia arriba.
Estadísticas descriptivas:
Ventas promedio más altas:
X360 (0.81M) y PS4 (0.80M) lideran, seguidos de cerca por PS3 (0.66M).
Estas plataformas destacan como las más rentables en términos generales.
Ventas más bajas:
PSP (0.13M) y PSV (0.12M) tienen las menores ventas promedio, reflejando su limitada popularidad.
Conclusión:
Plataformas líderes: X360, PS4, y PS3 son las más rentables en el período analizado.
Plataformas menos competitivas: PSP y PSV tienen un impacto marginal en ventas globales.
Recomendación: Focalizar campañas publicitarias en plataformas con alta variabilidad y éxito, como PS4 y X360, aprovechando su potencial de ventas por título.


- Mira cómo las reseñas de usuarios y profesionales afectan las ventas de una plataforma popular (tu elección). Crea un gráfico de dispersión y calcula la correlación entre las reseñas y las ventas. Saca conclusiones.


In [ ]:
# Elegir una plataforma popular para el análisis (ejemplo: PS4)
ps4_data = relevant_data[relevant_data['platform'] == 'PS4']

# Calcular correlaciones entre reseñas y ventas
correlation_critic_sales = ps4_data['critic_score'].corr(ps4_data['total_sales'])
correlation_user_sales = ps4_data['user_score'].corr(ps4_data['total_sales'])

# Crear gráficos de dispersión para críticas y ventas
plt.figure(figsize=(14, 6))

# Gráfico para reseñas de críticos
plt.subplot(1, 2, 1)
plt.scatter(ps4_data['critic_score'], ps4_data['total_sales'], alpha=0.6, color='blue')
plt.title("Relación entre Critic Score y Ventas (PS4)", fontsize=12)
plt.xlabel("Critic Score", fontsize=10)
plt.ylabel("Ventas Totales (millones)", fontsize=10)
plt.grid(alpha=0.3)

# Gráfico para reseñas de usuarios
plt.subplot(1, 2, 2)
plt.scatter(ps4_data['user_score'], ps4_data['total_sales'], alpha=0.6, color='green')
plt.title("Relación entre User Score y Ventas (PS4)", fontsize=12)
plt.xlabel("User Score", fontsize=10)
plt.ylabel("Ventas Totales (millones)", fontsize=10)
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

Correlación entre críticas y ventas:
Critic Score:

Correlación de 0.41, lo que indica una relación moderada positiva.
Los juegos con mejores críticas profesionales tienden a generar mayores ventas.
User Score:

Correlación de -0.03, lo que sugiere que no existe una relación significativa entre las reseñas de usuarios y las ventas.
Observaciones gráficas:
Critic Score:
Se observa una tendencia ascendente en el gráfico de dispersión, confirmando la influencia positiva de las críticas profesionales en las ventas.
User Score:
Los datos están dispersos sin un patrón claro, indicando que las ventas no están determinadas por las puntuaciones de los usuarios.
Conclusión:
Reseñas profesionales: Son un factor clave para el éxito comercial en la plataforma PS4.
Reseñas de usuarios: Parecen no ser un indicador confiable de ventas.
Recomendación: Invertir en la promoción de críticas positivas por parte de expertos para aumentar la visibilidad y las ventas de los juegos.


- Teniendo en cuenta tus conclusiones compara las ventas de los mismos juegos en otras plataformas.


In [ ]:
# Filtrar juegos que están disponibles en múltiples plataformas
multi_platform_games = relevant_data.groupby('name').filter(lambda x: len(x['platform'].unique()) > 1)

# Identificar un juego popular disponible en múltiples plataformas dentro de los datos relevantes
multi_platform_games_names = multi_platform_games['name'].value_counts().index[:1]  # Seleccionar el juego más común
selected_game = multi_platform_games_names[0]

# Obtener los datos del juego seleccionado
selected_game_data = relevant_data[relevant_data['name'] == selected_game]

# Crear un gráfico de barras para comparar ventas por plataforma
plt.figure(figsize=(10, 6))
plt.bar(selected_game_data['platform'], selected_game_data['total_sales'], color='skyblue', alpha=0.8)
plt.title(f"Comparación de Ventas por Plataforma para '{selected_game}'", fontsize=14)
plt.xlabel("Plataforma", fontsize=12)
plt.ylabel("Ventas Totales (millones de dólares)", fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Ventas por plataforma:
PlayStation 3 (PS3): Líder con 6.46 millones de dólares en ventas.

Xbox 360 (X360): Segunda posición con 4.22 millones.

PlayStation 4 (PS4): Genera ventas significativas con 3.01 millones.

Otras plataformas: Xbox One (XOne), PSV, PC, Wii, 3DS y PSP muestran menores contribuciones, aunque aún relevantes.

Observaciones clave:
Generaciones anteriores: PS3 y X360 dominan las ventas, mostrando su fuerte base de usuarios en 2014.

Plataformas recientes como PS4 y XOne: Aunque están en una etapa temprana, ya comienzan a acumular ventas importantes.

Plataformas portátiles y PC: Contribuyen con ventas mucho más bajas, reflejando su menor impacto para este título en particular.

Conclusión:
"FIFA 14" confirma que las plataformas con bases de usuarios establecidas (PS3 y X360) dominan las ventas, pero las nuevas generaciones (PS4 y XOne) comienzan a captar una proporción significativa del mercado. Esto refuerza la importancia de focalizar esfuerzos en plataformas emergentes mientras se mantienen estrategias para las más establecidas.

- Echa un vistazo a la distribución general de los juegos por género. ¿Qué se puede decir de los géneros más rentables? ¿Puedes generalizar acerca de los géneros con ventas altas y bajas?


In [ ]:
# Distribución de juegos por género
genre_distribution = relevant_data['genre'].value_counts()

# Ventas totales por género
genre_sales = relevant_data.groupby('genre')['total_sales'].sum().sort_values(ascending=False)

# Gráficos de distribución y ventas por género
plt.figure(figsize=(14, 6))

# Distribución de juegos por género
plt.subplot(1, 2, 1)
plt.bar(genre_distribution.index, genre_distribution.values, color='teal', alpha=0.75)
plt.title("Distribución de Juegos por Género", fontsize=14)
plt.xlabel("Género", fontsize=12)
plt.ylabel("Número de Juegos", fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Ventas totales por género
plt.subplot(1, 2, 2)
plt.bar(genre_sales.index, genre_sales.values, color='orange', alpha=0.75)
plt.title("Ventas Totales por Género", fontsize=14)
plt.xlabel("Género", fontsize=12)
plt.ylabel("Ventas Totales (millones de dólares)", fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

genre_distribution, genre_sales

Observaciones sobre la distribución:
Géneros más frecuentes:

Acción: Mayor número de juegos lanzados (1,495 títulos).
Misceláneos (Misc), Deportes y RPG: También tienen una representación significativa.
Géneros menos comunes:

Puzzle y Plataforma: Representan los géneros con menor cantidad de juegos lanzados.
Observaciones sobre las ventas:
Géneros más rentables:

Acción: Genera la mayor cantidad de ingresos (673.09 millones de dólares).
Shooter y Deportes: También son géneros líderes, con 479.74 y 328.39 millones respectivamente.
Géneros menos rentables:

Puzzle y Estrategia: Tienen las ventas totales más bajas, reflejando su menor atractivo comercial.
Generalización:
Géneros con ventas altas: Los juegos de Acción, Shooter y Deportes son los más rentables, destacándose por su popularidad y atractivo en audiencias amplias.
Géneros con ventas bajas: Los juegos de Puzzle y Estrategia tienden a ser más de nicho, con menor presencia en el mercado y ventas limitadas.

Paso 4. Crea un perfil de usuario para cada región
===============
Para cada región (NA, UE, JP) determina:

- Las cinco plataformas principales. Describe las variaciones en sus cuotas de mercado de una región a otra.

In [ ]:
# Calcular las ventas totales por plataforma para cada región
region_platform_sales = {
    "NA": relevant_data.groupby('platform')['na_sales'].sum().sort_values(ascending=False),
    "EU": relevant_data.groupby('platform')['eu_sales'].sum().sort_values(ascending=False),
    "JP": relevant_data.groupby('platform')['jp_sales'].sum().sort_values(ascending=False)
}

# Seleccionar las 5 plataformas principales para cada región
top_platforms_by_region = {
    region: sales.head(5) for region, sales in region_platform_sales.items()
}

# Calcular las cuotas de mercado por región
market_share_by_region = {
    region: (sales / sales.sum()) * 100 for region, sales in top_platforms_by_region.items()
}

print(top_platforms_by_region, market_share_by_region)

- Los cinco géneros principales. Explica la diferencia.

In [ ]:
# Calcular las ventas totales por género para cada región
region_genre_sales = {
    "NA": relevant_data.groupby('genre')['na_sales'].sum().sort_values(ascending=False),
    "EU": relevant_data.groupby('genre')['eu_sales'].sum().sort_values(ascending=False),
    "JP": relevant_data.groupby('genre')['jp_sales'].sum().sort_values(ascending=False)
}

# Seleccionar los 5 géneros principales para cada región
top_genres_by_region = {
    region: sales.head(5) for region, sales in region_genre_sales.items()
}

print(top_genres_by_region)

Diferencias entre regiones
Norteamérica y Europa:
Géneros de Acción, Shooter y Deportes dominan en ambas regiones.

RPG y Misc: Aunque menos populares, también tienen una representación significativa.
Japón:

RPG: Lidera significativamente con más del doble de ventas que el segundo género (Acción).
Géneros como Plataforma y Aventura tienen más relevancia en comparación con NA y EU.
Conclusión

NA y EU: Los géneros de acción y shooter dominan, reflejando una preferencia por experiencias intensas y competitivas.

Japón: Las preferencias se inclinan hacia RPG, mostrando un fuerte interés por juegos con historias profundas y elementos estratégicos.

- Si las clasificaciones de ESRB afectan a las ventas en regiones individuales.


In [ ]:
# Calcular las ventas totales por clasificación ESRB para cada región
region_rating_sales = {
    "NA": relevant_data.groupby('rating')['na_sales'].sum().sort_values(ascending=False),
    "EU": relevant_data.groupby('rating')['eu_sales'].sum().sort_values(ascending=False),
    "JP": relevant_data.groupby('rating')['jp_sales'].sum().sort_values(ascending=False)
}

# Normalizar las ventas para obtener proporciones por región
rating_market_share = {
    region: (sales / sales.sum()) * 100 for region, sales in region_rating_sales.items()
}

print(region_rating_sales, rating_market_share)

Observaciones clave
Norteamérica y Europa:

Los juegos clasificados como M (Mature) son los más rentables, seguidos de cerca por E (Everyone).
Clasificaciones como T (Teen) y E10+ también tienen una contribución relevante.
Japón:

Una gran proporción de ventas pertenece a juegos sin clasificación registrada (Unknown), reflejando una menor dependencia de ESRB.
Los juegos E (Everyone) y T (Teen) tienen más impacto que los clasificados como M (Mature).
Conclusión
NA y EU: Las clasificaciones ESRB influyen significativamente en las ventas, con un enfoque claro en juegos para audiencias maduras (M).
JP: Las clasificaciones tienen menor relevancia, y las ventas están más distribuidas entre géneros variados.

Paso 5. Prueba las siguientes hipótesis:
========

- Las calificaciones promedio de los usuarios para las plataformas Xbox One y PC son las mismas.

In [ ]:
from scipy.stats import ttest_ind, levene

# Filtrar datos para Xbox One y PC
xone_scores = relevant_data[relevant_data['platform'] == 'XOne']['user_score'].dropna()
pc_scores = relevant_data[relevant_data['platform'] == 'PC']['user_score'].dropna()

# Prueba de Levene para igualdad de varianzas
levene_stat, levene_p = levene(xone_scores, pc_scores)
equal_var = levene_p > 0.05  # Si p > 0.05, asumimos varianzas iguales

# Prueba de hipótesis (t-test)
t_stat, p_value = ttest_ind(xone_scores, pc_scores, equal_var=equal_var)

# Resultados
{
    "Levene Statistic": levene_stat,
    "Levene p-value": levene_p,
    "Supuesto de varianza igual": equal_var,
    "t-Statistic": t_stat,
    "p-value": p_value
}

No rechazamos la hipótesis nula de la prueba de Levene, concluyendo que las varianzas entre los grupos son iguales.

No rechazamos la hipótesis nula de la prueba t, concluyendo que las medias de los dos grupos no son significativamente diferentes.


- Las calificaciones promedio de los usuarios para los géneros de Acción y Deportes son diferentes.



In [ ]:
# Filtrar datos para los géneros Acción y Deportes
action_scores = relevant_data[relevant_data['genre'] == 'Action']['user_score'].dropna()
sports_scores = relevant_data[relevant_data['genre'] == 'Sports']['user_score'].dropna()

# Prueba de Levene para igualdad de varianzas
levene_stat, levene_p = levene(action_scores, sports_scores)
equal_var = levene_p > 0.05  # Si p > 0.05, asumimos varianzas iguales

# Prueba de hipótesis (t-test)
t_stat, p_value = ttest_ind(action_scores, sports_scores, equal_var=equal_var)

# Resultados
{
    "Levene Statistic": levene_stat,
    "Levene p-value": levene_p,
    "Supuesto de varianza igual": equal_var,
    "t-Statistic": t_stat,
    "p-value": p_value
}

El valor p es mucho menor que el nivel de significancia 𝛼 = 0.05

Esto implica que rechazamos la hipótesis nula de la prueba de Levene, concluyendo que las varianzas entre los grupos no son iguales.

Por lo tanto, en la prueba t de Student, utilizamos equal_var=False.

El valor p es mucho menor que 𝛼 = 0.05

Esto implica que rechazamos la hipótesis nula de la prueba t, concluyendo que las medias de los dos grupos son significativamente diferentes.

- Cómo formulaste las hipótesis nula y alternativa.



Hipótesis nula (H₀):
La hipótesis nula asume que no existe una diferencia significativa entre las medias de los grupos analizados.

Hipótesis alternativa (H₁):
La hipótesis alternativa propone que existe una diferencia significativa entre las medias de los grupos.

Prueba de Levene:
Objetivo: Evaluar si las varianzas entre los grupos (Xbox One y PC; Acción y Deportes) son iguales.
Si las varianzas no son iguales, se debe usar la versión de la prueba t que no asume igualdad de varianzas (equal_var=False).
Prueba t de Student (t-test):
Objetivo: Comparar las medias de dos grupos independientes para determinar si son significativamente diferentes.
Criterio de decisión:
Fijamos un nivel de significancia (𝛼) en 0.05.

Si el p-value de la prueba t es menor que 𝛼 , se rechaza la hipótesis nula, concluyendo que las medias son diferentes.

Si el p-value es mayor o igual a 𝛼, no se rechaza la hipótesis nula, indicando que no hay evidencia suficiente para afirmar que las medias son diferentes.


Paso 6. Escribe una conclusión general
==========

Principales Hallazgos
-----

1. Tendencias de Plataformas:

- PS4 y Xbox One son plataformas emergentes con potencial significativo en 2017.

- Consolas más antiguas como PS3 y X360 están en declive pero mantienen ventas relevantes.

2. Distribución por Géneros:

- Géneros como Acción, Shooter y Deportes son los más rentables a nivel global.

- En Japón, los RPG dominan las preferencias.

3. Impacto Regional:

- NA y EU muestran patrones similares con preferencia por plataformas de sobremesa.

- JP se inclina hacia consolas portátiles y géneros específicos como RPG y Aventura.

4. Reseñas y Ventas:

- Las críticas profesionales tienen un impacto positivo moderado en las ventas.

- Las puntuaciones de usuarios no muestran correlación significativa.

5. Hipótesis:

- No hay diferencia significativa entre las calificaciones de usuarios para Xbox One y PC.

- Existe una diferencia significativa entre las calificaciones de usuarios para los géneros de Acción y Deportes.